In [1]:
import pandas as pd
import os
import numpy as np


data_directory = "/home/skovtun/Python_projects/Kaggle/Single cell/Single_cell_data"
os.chdir(data_directory)
#print(os.getcwd())


In [2]:
pdata = pd.read_csv("./metadata.csv")

In [76]:
pdata.head()

,cell_id,day,donor,cell_type,technology
0,c2150f55becb,2,27678,HSC,citeseq
1,65b7edf8a4da,2,27678,HSC,citeseq
2,c1b26cb1057b,2,27678,EryP,citeseq
3,917168fa6f83,2,27678,NeuP,citeseq
4,2b29feeca86d,2,27678,EryP,citeseq


In [77]:
pdata.isnull().sum().sum()

0

In [79]:
nrows = pdata.shape[0]
for col in pdata.columns:
    if pdata[col].unique().size == nrows:
        print(f"Column {col} has all unique values")
    elif pdata[col].unique().size < 10:
        print(f"Column {col} had {pdata[col].unique().size} unique values.")
        print(pdata[col].value_counts())
    else:
        print (f"Column {col} has  {pdata[col].unique().size} values. It is {pdata[col].unique().size/nrows*100}% of all values.")
     

Column cell_id has all unique values
Column day had 5 unique values.
4     67111
3     64154
2     62250
7     56240
10    31773
Name: day, dtype: int64
Column donor had 4 unique values.
31800    74963
13176    73211
32606    72359
27678    60995
Name: donor, dtype: int64
Column cell_type had 8 unique values.
HSC       77535
hidden    55935
NeuP      42979
EryP      41479
MasP      34827
MkP       24021
MoP        3883
BP          869
Name: cell_type, dtype: int64
Column technology had 2 unique values.
multiome    161877
citeseq     119651
Name: technology, dtype: int64


In [3]:
# Because there is supposed to be  2 models, one for each technology, it makes sence to split pdata by technology
mult = pdata[pdata['technology'] == 'multiome'].drop('technology', axis=1)
cite = pdata[pdata['technology'] == 'citeseq'].drop('technology', axis=1)

In [4]:
# Looking at ways to split multiome data into chunks
mt = mult.groupby(['day','donor']).aggregate({'cell_type':'count'})

In [5]:
print(mt)

           cell_type
day donor           
2   13176       9558
    27678       8921
    31800       7089
    32606       7264
3   13176       8965
    27678       7950
    31800      11306
    32606       8544
4   13176       9407
    31800      10794
    32606      10933
7   13176       7466
    27678       7291
    31800       7656
    32606       6960
10  13176       8421
    27678       8790
    31800       7144
    32606       7418


In [16]:
mult_day2_donor1=[i.encode('UTF-8') for i in mult['cell_id'].loc[(mult['day'] == 2) & (mult['donor'] == 13176)]]
print(mult_day2_donor1[:5])


[b'cc1db26a8022', b'9cd4f248468c', b'891a65593475', b'9958bac2678b', b'35311a9bd7d5']


In [14]:
mult_day2_donor1.type

AttributeError: 'Series' object has no attribute 'type'

In [145]:
ct = cite.groupby(['day','cell_type']).aggregate({'cell_type': 'count'} )
ct.columns = ['cell_type_count']
ct = ct.reset_index()
ct['cell_type_by_day'] = ct['cell_type_count'] / ct.groupby('day')['cell_type_count'].transform('sum')*100


In [146]:
ct.pivot_table('cell_type_by_day', ['cell_type'], 'day')

day,2,3,4,7
cell_type,,,,
BP,0.523489,0.062069,0.091725,0.368482
EryP,16.605480,21.632772,24.693554,17.307478
HSC,48.596098,42.984410,32.665314,18.807459
MasP,8.895914,11.059184,15.979654,24.919046
MkP,9.633558,6.079083,6.459683,14.802546
MoP,0.271942,1.000402,1.264697,3.770425
NeuP,15.473520,17.182080,18.845373,20.024565


I would expect consistent distribution by cell types across all days. But in day 7 % of HSC dropped from 49% to 19%. Batch effect? Donor effect? Let's see if every donor has the same pattern.

In [121]:
ct = cite.groupby(['day','donor','cell_type']).aggregate({'cell_type': 'count'} )
ct.columns = ['cell_type_count']
ct = ct.reset_index()
ct['cell_type_by_day'] = ct['cell_type_count'] / ct.groupby(['day','donor'])['cell_type_count'].transform('sum')*100

In [122]:
ct.pivot_table('cell_type_by_day', ['donor','cell_type'], 'day')

day                      2          3          4          7
donor cell_type                                            
13176 BP          0.675342        NaN   0.141426   0.208478
      EryP       15.038709  18.709931  22.616382  14.940931
      HSC        51.556580  47.507523  36.629346  21.820709
      MasP        6.605172   8.412927  14.201532  20.681028
      MkP        12.040850   7.169959   6.517384  19.221682
      MoP         0.280020   0.758864   0.730701   2.376650
      NeuP       13.803327  17.440795  19.163229  20.750521
27678 BP          0.494917   0.046239   0.051073   0.752361
      EryP       17.589620  28.236745  29.443309  18.168721
      HSC        47.405029  34.694821  27.413177  13.766608
      MasP       10.486891  15.135635  17.709397  31.503121
      MkP         7.677903   5.332922   6.652196  12.934208
      MoP         0.214018   1.433416   1.391726   5.010405
      NeuP       16.131621  15.120222  17.339122  17.864575
31800 BP          0.464562   0.191724   0.059119   0.097229
      EryP       15.985706  19.651702  24.248694  22.897423
      HSC        48.576534  44.959259  33.165829  22.249230
      MasP        7.718880   9.793897  15.124643  16.755793
      MkP        11.375819   6.790222   6.985910  15.832118
      MoP         0.369267   1.294136   2.079023   4.310485
      NeuP       15.509232  17.319061  18.336782  17.857722
32606 BP          0.494917   0.028576   0.115656   0.427350
      EryP       17.589620  20.474353  23.110083  14.157706
      HSC        47.405029  43.963423  32.919777  17.231872
      MasP       10.486891  11.301615  17.053938  30.397022
      MkP         7.677903   4.943563   5.688151  11.152468
      MoP         0.214018   0.600086   0.767532   3.625586
      NeuP       16.131621  18.688384  20.344864  23.007996

So, this trend to see % of HSC diminishing is present in all donors, that makes me think it does depends on time, representing some biological processes. I read in discussion that donors were given some stimulants to bring more cells from Bone marrow to blood, it could be the reason.
Besides, In the Multiome, including cell type labels would constitute a data leak because the labels are based on the RNA.
So as a first step I would prefer not to include this information in both models. Maybe for citeseq I would include it later, or maybe create different models for each cell type.

In [123]:
day_cite = cite.groupby(['day','donor']).aggregate({'cell_id': 'count'})
day_cite.columns = ['cells_count']
day_cite = day_cite.reset_index()
day_cite['cells_by_day_%'] = day_cite['cells_count']/day_cite.groupby(['day'])['cells_count'].transform('sum')*100
# We see that day 10 is underrepresented
print("Percentage of cells from each donor for every day")
day_cite.pivot_table('cells_by_day_%',['donor'],'day')

Percentage of cells from each donor for every day


day,2,3,4,7
donor,,,,
13176,20.637025,27.905363,23.584512,26.780065
27678,25.413012,23.688342,21.769464,23.251573
31800,28.536950,22.852240,28.209690,22.968698
32606,25.413012,25.554055,26.436334,26.999665


We are seing some fluctuations but not as big as with cell types.

In [128]:
#Let's analyse mult dataframe
ctm = mult.groupby(['day','cell_type']).aggregate({'cell_id': "count"})
ctm.columns = ["cell_count"]
ctm = ctm.reset_index()
ctm['cell_type_%'] = ctm['cell_count']/ctm.groupby('day')['cell_count'].transform("sum")*100
ctm.pivot_table(['cell_type','cell_type_%'],"cell_type","day")

cell_count                                   cell_type_%             \
day               2        3       4       7        10          2          3    
cell_type                                                                       
BP             262.0    121.0   123.0    60.0      NaN    0.798002   0.329117   
EryP          3402.0   4699.0  6054.0  2980.0      NaN   10.361842  12.781178   
HSC          10757.0  11167.0  9116.0  3621.0      NaN   32.763767  30.373997   
MasP          2175.0   3549.0  5228.0  5785.0      NaN    6.624635   9.653203   
MkP           3394.0   2886.0  3495.0  3446.0      NaN   10.337476   7.849857   
MoP            258.0    498.0   462.0   843.0      NaN    0.785819   1.354549   
NeuP          3663.0   5895.0  6656.0  5347.0      NaN   11.156798  16.034272   
hidden        8921.0   7950.0     NaN  7291.0  31773.0   27.171662  21.623827   

                                        
day               4          7      10  
cell_type                               
BP          0.395066   0.204269    NaN  
EryP       19.444980  10.145372    NaN  
HSC        29.279887  12.327648    NaN  
MasP       16.791932  19.694958    NaN  
MkP        11.225670  11.731863    NaN  
MoP         1.483908   2.869983    NaN  
NeuP       21.378557  18.203793    NaN  
hidden           NaN  24.822116  100.0

In this dataframe there are cells without identifications, so theese values skews percentages, so to check if the
tendencies we saw in cite dataframe are present, we have to get rid of "hidden" category.
One more irregularity: there are no cells marked as "hidden" on day 4.

In [133]:
mult_r = mult[mult['cell_type']!='hidden']
ctmr = mult_r.groupby(['day','cell_type']).aggregate({'cell_id': 'count'})
ctmr.columns = ['cell_counts']
ctmr = ctmr.reset_index()
ctmr.head()

,day,cell_type,cell_counts
0,2,BP,262
1,2,EryP,3402
2,2,HSC,10757
3,2,MasP,2175
4,2,MkP,3394


In [148]:
ctmr['cell_type_in_a_day%'] = ctmr['cell_counts']/ctmr.groupby('day')['cell_counts'].transform('sum')*100

In [135]:
ctmr.head()

,day,cell_type,cell_counts,cell_type_in_a_day%
0,2,BP,262,0.010957
1,2,EryP,3402,0.142278
2,2,HSC,10757,0.449877
3,2,MasP,2175,0.090962
4,2,MkP,3394,0.141943


In [149]:
ctmr.pivot_table(['cell_type_in_a_day%'], 'cell_type','day')

cell_type_in_a_day%                                 
day                         2          3          4          7
cell_type                                                     
BP                   1.095730   0.419920   0.395066   0.271715
EryP                14.227761  16.307479  19.444980  13.495154
HSC                 44.987663  38.754121  29.279887  16.397971
MasP                 9.096232  12.316502  16.791932  26.197808
MkP                 14.194304  10.015617  11.225670  15.605471
MoP                  1.079001   1.728267   1.483908   3.817589
NeuP                15.319309  20.458095  21.378557  24.214292

In [147]:
ct.pivot_table('cell_type_by_day', ['cell_type'], 'day')

day,2,3,4,7
cell_type,,,,
BP,0.523489,0.062069,0.091725,0.368482
EryP,16.605480,21.632772,24.693554,17.307478
HSC,48.596098,42.984410,32.665314,18.807459
MasP,8.895914,11.059184,15.979654,24.919046
MkP,9.633558,6.079083,6.459683,14.802546
MoP,0.271942,1.000402,1.264697,3.770425
NeuP,15.473520,17.182080,18.845373,20.024565


I would say these two tables showes consistent picture that there were some processes between days 2 and 7 that influenced such dramatic decrease in proportion of HSC cells vs increase of MasP cells